# Football Wages Web Scraping 2

This scrapes the web for football wage data. It loops through the urls for the top 5 leagues, within each url it loops through the last 10 full seasons (14/15 - 23/24). It saves the data as CSVs in the relevant folder for each league.

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time

In [31]:
#different urls for each league
PL = "https://fbref.com/en/comps/9/wages/Premier-League-Wages"      #premier league
SA = "https://fbref.com/en/comps/11/wages/Serie-A-Wages"            #serie a
LL = "https://fbref.com/en/comps/12/wages/La-Liga-Wages"            #la liga
BL = "https://fbref.com/en/comps/20/wages/Bundesliga-Wages"         #bundesliga
LU = "https://fbref.com/en/comps/13/wages/Ligue-1-Wages"            #ligue 1
urls = [PL, SA, LL, BL, LU]
leagues = ['PL','SA','LL','BL','LU']
seasons = ['23-24','22-23','21-22','20-21','19-20','18-19','17-18','16-17','15-16','14-15']

In [55]:
for i in range(len(urls)):
    url = urls[i]                    #loop throuh each league using their urls

    #opens the url with the chrome web driver
    driver = webdriver.Chrome()
    driver.get(url)

    #rejects all cookies when page opens up
    no_cookies = driver.find_element(By.XPATH, "//button[@class=' osano-cm-denyAll osano-cm-buttons__button osano-cm-button osano-cm-button--type_denyAll ']")
    no_cookies.click()

    #waits 1 second to allow page to load
    time.sleep(1)

    for j in range(len(seasons)):
        
        #clicks previous season button to go back to previous season
        prev_season = driver.find_element(By.XPATH, "//a[@class='button2 prev']")
        prev_season.click()

        time.sleep(1)

        if j == 3:
            popup_close = driver.find_element(By.XPATH, "//div[@id='modal-close']")    #closes pop-up that appears
            popup_close.click()
            time.sleep(1)

        #defines 'soup' as the pages html and 'table' as the specific html for the table of data
        soup = bs(driver.page_source, "html.parser")
        table = soup.find("table", {"id": "squad_wages"})

        #pulls the table headers from the html
        table_titles = table.find_all("th", {"scope": "col"})
        titles = [title.text.strip() for title in table_titles]

        #creates a data frame with the table headers
        df = pd.DataFrame(columns = titles)

        all_data = []

        for t in table:
            rows = table.find("tbody").find_all("tr")           #finds the rows of data using the tags 'tbody' and 'tr'
            for row in rows:
                cols = row.find_all(["th","td"])                #finds the individual data using the tags 'th' and 'td'
                cols = [col.text.strip() for col in cols]       #cleans the data using strip()
                all_data.append(cols)
                length = len(df)                                #adds the data to the dataframe
                df.loc[length] = cols


        #deletes duplicate rows
        df = df.drop_duplicates()

        #saves file as league + season in that leagues folder e.g. for the first one 'PL 23-24.csv' in the PL folder within the Football Wages folder.
        folder = r"C:\Users\mattc\OneDrive\Documents\Data\Football Wages\\" + leagues[i] + "\\"
        filename = leagues[i] + " " + seasons[j] + ".csv"
        df.to_csv(folder + filename, index=False)


#closes the web driver browser
driver.quit()
driver = None

time.sleep(1)